<div>
<img src="img/logo.png" width="500"/>
</div>
<h1>Introduction to u2py module</h1>


## Importing and using the u2py module

Note that the Python Version of Python shipped with the Rocket U2 Databases has a u2.pth file, which defines where to find the u2py.py module.  If you are not able to import the u2py module, please verify that you are running the version configured for the Rocket MultiValue Database.

Note that you can check that the OS Environment PATH variable, has the path to the U2 configured Python Version first, prior to executing python, or you can check from within python. 


In [1]:
import sys
import os
pythonPath = sys.executable
pythonDir = os.path.dirname(pythonPath)
pthPath = os.path.join(pythonDir, "u2.pth")
print("Checking for " + str(pthPath))
try:
    with open(pthPath) as f:
        lines = f.readlines()
    print(lines)
except Exception as e:
    print("Could not read the u2.pth file.")
    print("Check which version of Python you are using.")
    print("")
    print("The Python you are running is " + sys.version + " and is found in " + pythonDir)


Checking for c:\U2\ud83\Python\u2.pth
['C:\\U2\\ud83\\bin\n', 'C:\\U2\\ud83\\XDEMO\\PP\n']


## Navigating to the XDEMO account 

After you import the u2py module, you can determin the path of the U2BIN directory, and use that to navigate to the XDEMO account.

In [5]:
u2Home = os.path.abspath(os.path.join(pythonDir, os.pardir)) 
print("The U2HOME PATH is " + u2Home)
# Change the path to you local XDEMO account below:
xdemoPath = os.path.join(u2Home, "XDEMO")
os.chdir(xdemoPath)
print(os.getcwd())
import u2py

The U2HOME PATH is c:\U2\ud83
c:\U2\ud83\XDEMO


#### Note if you are having problems with any of these examples, please see the [Troble shooting u2py](Trouble_Shooting_u2py.ipynb#Main)

## Working with the u2py module

The u2py.py module for Python, allows Python developers to access the Rocket MultiValue Database directly from the server.  

u2py allows you to do the following
1. [Handle U2 dynamic arrays](#u2py.DynArray)
1. [Read/write U2 files](#u2py.File)
1. [Run U2 ECL/TCL commands](#u2py.run)
1. [Manage U2 SELECT list](#u2py.List)
1. [Call U2BASIC catalogued subroutines](#u2py.Subroutine)
1. Control U2 transactions

Note that the the above order of the topics is different than how they are displayed in the help(u2py), that is because I plan on using somesomthing I show in earler sections in teaching the later sections.  While you are free to jump around, I may not reiterate objects and methods I expect you to know.

<a id='u2py.DynArray'></a>
### Handle U2 dynamic arrays

The u2py.DynArray is an object that encapsulates many functions and methods that are familure to the MultiValue Developer, while am no going to review them all in this note book, the following help command displays a complete list.

In [6]:
help(u2py.DynArray)

Help on class DynArray in module u2py:

class DynArray(_u2py._DynArray)
 |  DynArray(*args)
 |  
 |  DynArray([arg]) -> new U2PY DynArray object -- to support manipulation of MV delimited data, stores data internally as bytes
 |  
 |  DynArray() -- an empty DynArray object
 |  DynArray(arg) -- an DynArray object with its value set to arg's bytes/string representation
 |  
 |  DynArray is iterable. The iterator returns a Python tuple object with two items (v, d) : 
 |     v is the dynamic array element extracted;
 |     d is the system delimiter found: 0 End of dynamic array, 1 Item mark, 2 Field mark, 3 Value mark, 4 Subvalue mark, 5 Text mark.
 |  
 |  Method resolution order:
 |      DynArray
 |      _u2py._DynArray
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  extract(self, *args, **kwargs)
 |      D.extract(field#

#### How to create a u2py Dynamic Array Object

For this example, I am going to create Dynamic Array Object with two attributes.  Where Attribute 1 will be equle to "ONE" and Attribute 2 will be equle to "TWO"


In [7]:
# Example 1 - Starting with an empty array
rec1 = u2py.DynArray()
rec1.insert(1, "One")
rec1.insert(2, "Two")
print("looking at the object we see:")
print(type(rec1))
print()
print("Converting the Object from example 1 to a string we see")
rec_str = str(rec1)
print(rec_str)

# Example 2 - Take a Dynamic Array String and marshel it into an Object.
rec2 = u2py.DynArray(rec_str)
print("Converting the Object from example 2 to a string we see")
rec_str = str(rec2)
print(rec_str)
# Example 3 - pass in a Python List
pyList = ["One", "Two"]
rec3 = u2py.DynArray(pyList)
print("Converting the Object from example 3 to a string we see")
rec_str = str(rec3)
print(rec_str)


looking at the object we see:
<class 'u2py.DynArray'>

Converting the Object from example 1 to a string we see
One�Two
Converting the Object from example 2 to a string we see
One�Two
Converting the Object from example 3 to a string we see
One�Two


Note that we can also compare the items to make sure they represent the same data.

In [8]:
if rec1==rec2 and rec2==rec3:
    print("All three represnet the same data")

#### Returning the Dynamic Array as a Python List

In the previous example, one of the methods we used to create the dyanmic array was by passing in a Python List.  in addition to passing a list into the u2py.DynArray Object, you can call a function to return the data as a nested Python List.



In [9]:
the_list = [ "att-1", ["att-2, Value-1", "att-2, Value-2"], [["att-3, Value-1, SubValue-1", "att-3, Value-1, SubValue-2" ],["att-3, Value-1, SubValue-2", "att-3, Value-2, SubValue-2" ]]]
dynArrayObj = u2py.DynArray(the_list)
print("The String representation look like:")
print(str(dynArrayObj))
as_list = dynArrayObj.to_list()
from pprint import pprint
pprint(as_list)

The String representation look like:
att-1�att-2, Value-1�att-2, Value-2�att-3, Value-1, SubValue-1�att-3, Value-1, SubValue-2�att-3, Value-1, SubValue-2�att-3, Value-2, SubValue-2
['att-1',
 ['att-2, Value-1', 'att-2, Value-2'],
 [['att-3, Value-1, SubValue-1', 'att-3, Value-1, SubValue-2'],
  ['att-3, Value-1, SubValue-2', 'att-3, Value-2, SubValue-2']]]


In the previous example we displayed the data as both a string, with delimeters, and a Python list.  Note that in the string representations of the
dynamic array object, the data is seperated by delimeters.

In [10]:
print(str(dynArrayObj))
print()
print("Note that the delimites are displayed as:")
print("u2py.IM" +  " - " + str(u2py.DynArray(u2py.IM)))
print("u2py.FM" +  " - " + str(u2py.DynArray(u2py.FM)))
print("u2py.VM" +  " - " + str(u2py.DynArray(u2py.VM)))
print("u2py.SM" +  " - " + str(u2py.DynArray(u2py.SM)))
print("u2py.TM" +  " - " + str(u2py.DynArray(u2py.TM)))

att-1�att-2, Value-1�att-2, Value-2�att-3, Value-1, SubValue-1�att-3, Value-1, SubValue-2�att-3, Value-1, SubValue-2�att-3, Value-2, SubValue-2

Note that the delimites are displayed as:
u2py.IM - �
u2py.FM - �
u2py.VM - �
u2py.SM - �
u2py.TM - �


##### By default the delimeters are set as follows:

uopy property             | Description.............. |                    Character
:-----------------------|:--------------------------:|--------------------------:
IM | Item Mark | b'\xff'
FM | Field/Attribute Mark|b'\xfe'
VM | Value Mark|b'\xfd'
SM | SubValue Mark|b'\xfc'
TM | Text Mark|b'\xfb'

<b>Rocket Software Support strongly recommends that the Python developer use the u2py properties rather than default binary character delimeters.</b>

Note that this is due to the fact that changes of codepages and languages used can change the values of the delimetors.


#### Accessing the same sub-value from the dynamic array object or the list

To retrieve a specific Attribute, Value or SubValue from a dynamic array object use the u2py.DynArray.extract method.

In [12]:
help(u2py.DynArray.extract)

Help on function extract in module u2py:

extract(self, *args, **kwargs)
    D.extract(field#, [value#, [subvalue#]]) -> new DynArray object -- extract a field, a value, or a subvalue from the dynamic array



Example:  Getting the 2nd value of the 2nd attribute


In [13]:
results = dynArrayObj.extract(2,2)
print(results)

att-2, Value-2


This is different than how a Python list works.  The first element of a python list is 0, and dynamic arrays with values and sub-values is
marshedled into a nested list. 

So, we can use the following to extract the same data.

In [14]:
result2 = dynArrayObj.to_list()[1][1]
print(result2)

att-2, Value-2


### Iterating over the u2py.DynArray Object

DynArray is iterable. The iterator returns a Python tuple object with two items (v, d) :
     v is the dynamic array element extracted;
     d is the system delimiter found: 0 End of dynamic array, 
                                      1 Item mark, 
                                      2 Field mark, 
                                      3 Value mark, 
                                      4 Subvalue mark, 
                                      5 Text mark.
     
So, it is possible to loop through the object and extract the data.


In [15]:
for data, delim in dynArrayObj:
    print(data, delim)

att-1 2
att-2, Value-1 3
att-2, Value-2 2
att-3, Value-1, SubValue-1 4
att-3, Value-1, SubValue-2 3
att-3, Value-1, SubValue-2 4
att-3, Value-2, SubValue-2 0


<a id='u2py.File'></a>
## Read/write U2 files

#### Prior to reading and/or writing to a Rocket MultiValue file, you first have to instantiate a u2py.File Object.


In [16]:
try:
    membersFile = u2py.File("MEMBERS")
except u2py.U2Error as e:
    print("Something major went wrong with the Jupyter notebook or the XDEMO account.")
membersFile

<u2py.File name='MEMBERS' status='opened'>

Note that since this notebook is expected to run from the XDMEO account, the MEMBERS file is expected to exist.  We put the u2py.File command in try-except,
because it is a good programing practice.  Please see the [u2py File_Best Practices](u2py_File_Best_Practices.ipynb#bp_u2py_File) for mor information.


### Reading from the u2py.file Object

You can get help on the u2py.File.read method by calling help on the method.

In [17]:
help(membersFile.read)

Help on method read in module u2py:

read(*args) method of u2py.File instance
    F.read(recordid, [lockflag]) -> new DynArray object -- read a record in the file
    
    lockflag is either 0 (default), or [LOCK_EXCLUSIVE or LOCK_SHARED] [ + LOCK_WAIT].



#### For a codding example, I am going to read an Item that should exist in the MEMBERS file of the XDEMO database.

In [18]:
rec = membersFile.read("0110")
# note that if successful the read will return a u2py.DynArray object
rec
print(str(rec))

Johnson�Anita�Y�Mrs�50 Portobello Road�New Glasgow�NE�01870�F�5139��9795570025�ajohnson@mv.rs.com�monkey�U2FsdGVkX18UefufEjOFs2GUdzdF3Y89ZYiosyRzt+8=
�ajohnson.jpg�A�9723122312945378�5700222144699144�9254766688143698�4699144669214769�U2FsdGVkX1/Hy9eK3j1yUEM5MtJ+XOsPGnGl95FzkitbBbE6eY4tX5ENetHMxlwj
�U2FsdGVkX1+TB/WIrJXzzZUnAojZoAcqvyumYKmQ4VDs70xVrnC59fsa+a/K9Q1Y
�U2FsdGVkX1+XOujSPxfiuQTW/QNsfgX7R7Rp7Z1e6n2mi0ebK2DYS5r4M3oG+MRF
�U2FsdGVkX1/bVAxPZZ1StQUcib8ZLI1SarQXAUImDYUuQz4qe4LGRVhwJPRgskvx
�AMEX�V�MC�MC�18019�17989�16834�17930�025�570�925�032


#### Next example shows what happens if the item does not exist.

In [19]:
badrec = membersFile.read("notavalidid")

U2Error: (30001) Record not found

As you see, when the item is not found, an exception is raised.  We can handle this in python with a try/except block

In [20]:
try:
    badrec = membersFile.read("notavalidid")
except u2py.U2Error as e:
    badrec = u2py.DynArray()
    print("caught an execptiom, badrec set to an empty dynamic array")

caught an execptiom, badrec set to an empty dynamic array


### Writing an item with the u2py.File.write method

You can see that the method is similar to the read, with the exception of the need to pass the u2py.DynArray to write to the method.

In [21]:
help(membersFile.write)


Help on built-in function write:

write(...) method of u2py.File instance
    F.write(recordid, record, [lockflag]) -> None -- write a record to the file
    
    lockflag is either 0, LOCK_RETAIN, LOCK_WAIT, LOCK_RETAIN + LOCK_WAIT.



For our example we will be creating a new item with an ID of "NEW", note that we will be deleting this item later in this notebook

In [22]:
membersFile.write("NEW", rec)
# Lets read it back in and see that it worked.
newrec = membersFile.read("NEW")
if newrec == rec:
    print("They match")
    print(str(newrec))
else:
    print("something went wrong with the write or read")

They match
Johnson�Anita�Y�Mrs�50 Portobello Road�New Glasgow�NE�01870�F�5139��9795570025�ajohnson@mv.rs.com�monkey�U2FsdGVkX18UefufEjOFs2GUdzdF3Y89ZYiosyRzt+8=
�ajohnson.jpg�A�9723122312945378�5700222144699144�9254766688143698�4699144669214769�U2FsdGVkX1/Hy9eK3j1yUEM5MtJ+XOsPGnGl95FzkitbBbE6eY4tX5ENetHMxlwj
�U2FsdGVkX1+TB/WIrJXzzZUnAojZoAcqvyumYKmQ4VDs70xVrnC59fsa+a/K9Q1Y
�U2FsdGVkX1+XOujSPxfiuQTW/QNsfgX7R7Rp7Z1e6n2mi0ebK2DYS5r4M3oG+MRF
�U2FsdGVkX1/bVAxPZZ1StQUcib8ZLI1SarQXAUImDYUuQz4qe4LGRVhwJPRgskvx
�AMEX�V�MC�MC�18019�17989�16834�17930�025�570�925�032


This simmple example wrote the dynamic arry to the file, read it back in and then compared what was weitten to what was read.  The last line prints
the string representation of the u2py.DynArray object. 


<a id='u2py.run'></a>
## Run U2 ECL/TCL commands

The u2py.Command object is used to execute a command as you would if you were at the ECL/TCL Prompt.  

In [23]:
help(u2py.Command)

Help on class Command in module u2py:

class Command(_u2py._Command)
 |  Command(cmdtext) -> new Command object -- to support the execution of U2 commands
 |  
 |  cmdtext -- ECL/TCL command text.
 |  
 |  Method resolution order:
 |      Command
 |      _u2py._Command
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  run(self, capture=False)
 |      run([capture=False]) -> None -- run the TCL/ECL command, similar to running a command from U2 prompt
 |      if capture is True, return the output of the command as a string
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from _u2py._Command:
 |  
 |  __init__

## There are a few options on how to instantiate the object and run the command.

#### Note that due to the nature of Jupyter-Notebooks, and u2py, the run code can not be interactive for these examples

### Option 1: Instantiate the object and execute the command by calling the run method:
    a = u2py.run("WHO", capture=True)
    a
    '5 XDEMO From ROCKET1\\mrajkowski\n'

### Option 2: Instantiate the object and execute the command by calling the run method without capturing the output:
    u2py.run("WHO")
    5 XDEMO From ROCKET1\mrajkowski

### Option 3: Option 2: Instantiate the object then call the run method:
    cmd = u2py.Command("WHO")
    a = cmd.run(capture=True)
    a
    '5 XDEMO From ROCKET1\\mrajkowski\n'            

### Option 4: Option 2: Instantiate the object then call the run method without capturing the output:
    cmd = u2py.Command("WHO")
    cmd.run()
    5 XDEMO From ROCKET1\mrajkowski

<a id='u2py.run'></a>Note that if you assign the output of the run method to a variable, and you do not have capture set to True,
the variable will be set to the NoneType.

    a = cmd.run()
    5 XDEMO From ROCKET1\mrajkowski
    type(a)
    <class 'NoneType'>

<a id='u2py.List'></a>
## Manage U2 SELECT list

This object allows the developer to access the active select lists from the Rocket U2 Database, or build an active select list from the file object, the file index, or by retrieving a select list from a previously saved list.

In [24]:
import u2py
file = u2py.File("STATES")
ids  = u2py.List(9, file)
as_python_list = ids.readlist().to_list()
print(str(as_python_list))

['AL', 'AR', 'CA', 'DC', 'IA', 'ID', 'LA', 'MI', 'MO', 'NE', 'NH', 'SC', 'TN', 'CT', 'GA', 'HI', 'IN', 'MA', 'MD', 'MS', 'NC', 'OH', 'OK', 'PA', 'SD', 'TX', 'UT', 'VA', 'WI', 'AK', 'AZ', 'CO', 'DE', 'FL', 'IL', 'KS', 'KY', 'ME', 'MN', 'MT', 'ND', 'NJ', 'NM', 'NV', 'NY', 'OR', 'RI', 'VT', 'WA', 'WV', 'WY']


Note that we needed to provide a active list number, in this case I used 9, to provide a place for the 
Rocket MultiValue database to store the select list.

I then used the readlist method from the u2py.List Object to read in all of the items at once.  While it is 
possible to iterate over the entire contents of the object, each iteration will need to interact with MultiValue.  It is quicker to read in all the item ids at once.  ( That is if you do not expect a Rocket MultiValue command or BASIC program to access the same list.

<a id='u2py.Subroutine'></a>
## Call U2BASIC catalogued subroutines

In the Rocket MultiValue DataBases, a subroutine is a BASIC program that has as a SUBROUTINE statement as its first noncommented line of code.

Note that all subroutines must be cataloged using the ECL/TCL CATALOG command before 
being called. For more information about the CATALOG command, see the Rocket U2 Documentation for the database you are using.

To instantiate a u2py.Subroutine object, you must provide the name of the subroutine, as cataloged, and the number are parameters the subroutine requires.

sub = Subroutine(name, pnum)

Consider the following subroutine:
: CT JBP SIMPLE_SUBROUTINE

#####     SIMPLE_SUBROUTINE
SUBROUTINE SIMPLE_SUBROUTINE( OUTVAL, INVAL )</P>
OUTVAL = "Hello ":INVAL</P>
RETURN</P>

The following is how we would call the subroutine:


In [26]:
sub = u2py.Subroutine("SIMPLE_SUBROUTINE", 2)
sub.args[0] = ""
sub.args[1] = "Mike"
sub.call()
print(sub.args[0])

Hello Mike
